# Boston Housing Clustering

In [ ]:
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import StringIndexer, VectorAssembler, BucketedRandomProjectionLSH, MinHashLSH
from pyspark.ml.clustering import KMeans, BisectingKMeans
from pyspark.sql.session import SparkSession, Row
from pyspark.sql.functions import desc, expr
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder
       .appName("ChurnDataPreprocessing")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

## Data Preparation

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

### KMeans 

In [ ]:
km = KMeans (seed=12345, featuresCol="features",predictionCol="prediction")   

### Build Param Grid

In [ ]:
paramGrid = ParamGridBuilder().addGrid(km.k, [2, 3, 4, 5, 6 ]) \
				                 .addGrid(km.maxIter, [10, 100 ]) \
                                 .build()

### Build the pipeline

In [ ]:
pipeline = Pipeline(stages = [assembler, km ])

### Build the evaluator

In [ ]:
evaluator = ClusteringEvaluator()

### Build the cross validator

In [ ]:
 cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=2, parallelism=2)

### Train the model 

In [ ]:
cvModel = cv.fit(df)

### Find the best model

In [ ]:
kmModel = cvModel.bestModel.stages[1]
print(kmModel.explainParams())
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
   print(center)

## Test the model

In [ ]:
predictions = cvModel.transform(df)
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " , silhouette)

## Bisecting k-means

### Build BisectingKMeans

In [ ]:
bkm = BisectingKMeans(seed=12345, featuresCol="features",predictionCol="prediction")   

### Build pararm grid

In [ ]:
paramGridBkm = ParamGridBuilder().addGrid(km.k, [2, 3, 4, 5, 6 ]) \
				                 .addGrid(km.maxIter, [10, 100 ]) \
                                 .build()

### Build pipeline

In [ ]:
pipelineBkm =  Pipeline(stages = [assembler, km ])

### Build Cross Validator

In [ ]:
cvbkm = CrossValidator(estimator=pipelineBkm,evaluator=evaluator,estimatorParamMaps=paramGridBkm, numFolds=2,parallelism=2)

### Train the model

In [ ]:
cvModebkml = cvbkm.fit(df)

### Find the best model

In [ ]:
kmModelbkm = cvModebkml.bestModel.stages[1]
print(kmModelbkm.explainParams())
centers = kmModelbkm.clusterCenters()
print("Cluster Centers: ")
for center in centers:
   print(center)

In [ ]:
### Test the model

In [ ]:
predictions = cvModebkml.transform(df)
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " , silhouette)

In [ ]:
spark.stop()